In [1]:
# Importing required libraries
import numpy as np
from time import time

# Defining variables
x_Bound = [0., 20.]   # xmin, xmax
y_Bound = [0., 10.]    # ymin, ymax
N, M = 201, 101        # mesh in x, mesh in y

# Make sure the PARA.F is uploaded to the colab

# DFIB
Cylinder_Center = [6., 5.]  # x, y
Cylinder_Radius = 0.5
uSolid = [0., 0.]   # u, v

In [2]:
# Mesh Code
length = x_Bound[1] - x_Bound[0]
height = y_Bound[1] - y_Bound[0]

dxx = length / N
dyy = height / M

# Creating XX.dat file
with open('XX.DAT', 'w') as f:
    for i in range(-1, N+2):
        x_i = dxx * i + x_Bound[0]
        f.write(f'{x_i} {i}\n')

# Creating YY.dat file
with open('YY.DAT', 'w') as f:
    for j in range(-1, M+2):
        y_j = dyy * j + y_Bound[0]
        f.write(f'{y_j} {j}\n')

# Creating mesh.dat file
with open('mesh.dat', 'w') as f:
    f.write('VARIABLES=X,Y\n')
    f.write(f'ZONE I={N}, J={M}\n')
    for j in range(1, M+1):
        for i in range(1, N+1):
            f.write(f'{dxx*i + x_Bound[0]} {dyy*j + y_Bound[0]}\n')

In [3]:
class Main_Class():
    def __init__(self, N, M, Cylinder_Center, Cylinder_Radius, uSolid):
        # M & N are always transfered just to make it short (N instead of self.N)
        # Conversion:
        #	-1 to N+1	--> nx = N+3, ny = M+3
        #	-1 to N 	--> nx = N+2
        #	0  to N		--> nx = N+1
        # Example N = 2 --> n(-1 to 3) = 5

        # File Names
        self.x_Mesh_File = "XX.DAT"
        self.y_Mesh_File = "YY.DAT"
        self.Parameter_File = "PARA.F"

        # Main Variables
        self.U = np.zeros((N + 3, M + 3))
        self.V = np.zeros((N + 3, M + 3))
        self.P = np.zeros((N + 3, M + 3))

        # Derivatives Vars
        self.DX = np.zeros(N + 3)
        self.DX1 = np.zeros(N + 3)
        self.DY = np.zeros(M + 3)
        self.DY1 = np.zeros(M + 3)

        # DFIB
        self.Eta = np.zeros((N + 2, M + 2))
        self.Fx = np.zeros((N + 3, M + 3))
        self.Fy = np.zeros((N + 3, M + 3))
        self.USolid = uSolid[0]
        self.VSolid = uSolid[1]
        self.Cylinder_Center = Cylinder_Center
        self.Cylinder_Radius = Cylinder_Radius

    def Load_Input(self, N, M):
        self.X = np.float64(np.loadtxt(self.x_Mesh_File, usecols=(0)))
        self.Y = np.float64(np.loadtxt(self.y_Mesh_File, usecols=(0)))
        nx = len(self.X)
        ny = len(self.Y)

        for i in range(nx - 1):
            self.DX[i] = self.X[i + 1] - self.X[i]
        for i in range(nx - 2):
            self.DX1[i] = 0.5 * (self.X[i + 2] - self.X[i])

        for j in range(ny - 1):
            self.DY[j] = self.Y[j + 1] - self.Y[j]
        for j in range(ny - 2):
            self.DY1[j] = 0.5 * (self.Y[j + 2] - self.Y[j])

        TempT = np.loadtxt(self.Parameter_File, delimiter=',', max_rows=1)
        self.TTT, self.OT, self.DT, self.FT = int(TempT[0]), int(TempT[1]), TempT[2], int(TempT[3])
        self.ITER, self.RE, self.OMEGA, self.RERUN = int(TempT[4]), TempT[5], TempT[6], int(TempT[7])

        self.Time = 0.
        self.U[:,:] = 1.
        self.V[:,:] = 0.
        self.P[:,:] = 0.

        self.VOS(N, M, self.Cylinder_Center, self.Cylinder_Radius)

    def VOS(self, N, M, Cylinder_Center, Cylinder_Radius):
        for j in range(2, M + 2):
            for i in range(2, N + 2):
                Cell_Center = [(self.X[i] + self.X[i + 1]) / 2.,
                               (self.Y[j] + self.Y[j + 1]) / 2.]
                Cell_Size   = [(self.X[i + 1] - self.X[i]),
                               (self.Y[j + 1] - self.Y[j])]
                Distance = np.sqrt(np.square(Cell_Center[0] - Cylinder_Center[0]) +
                                   np.square(Cell_Center[1] - Cylinder_Center[1]))
                Cell_Diagonal = np.sqrt(np.square(Cell_Size[0]) + np.square(Cell_Size[1]))

                if Distance > Cylinder_Radius:
                    self.Eta[i, j] = 0.
                else:
                    self.Eta[i, j] = 1.

                if np.abs(Distance - Cylinder_Radius) < Cell_Diagonal:
                    Subx = np.linspace(self.X[i], self.X[i + 1], 100)
                    Suby = np.linspace(self.Y[j], self.Y[j + 1], 100)
                    self.Eta[i, j] = 0
                    for ii in range(99):
                        for jj in range(99):
                            Sub_Center = [(Subx[ii] + Subx[ii + 1]) / 2.,
                                          (Suby[jj] + Suby[jj + 1]) / 2.]
                            Distance_Sub = np.sqrt(np.square(Sub_Center[0] - Cylinder_Center[0]) +
                                                   np.square(Sub_Center[1] - Cylinder_Center[1]))
                            if (Distance_Sub < Cylinder_Radius):
                                self.Eta[i, j] += 1. / 100 / 100

    def Load_Vel_Boundary(self, N, M):
        # Right
        self.U[N + 1, 1:M+2] = self.U[N, 1:M+2]
        self.V[N + 1, 1:M+2] = self.V[N, 1:M+2]
        self.V[N + 2, 1:M+2] = self.V[N + 1, 1:M+2]

        # Left
        self.U[1, 1:M+2] = 1.
        self.V[1, 1:M+2] = -self.V[2, 1:M+2]
        self.U[0, 1:M+2] = 1.
        self.V[0, 1:M+2] = self.V[1, 1:M+2]

        # Bot
        self.U[2:N+2, 1] = 2. -self.U[2:N+2, 2]
        self.V[2:N+2, 1] = 0.
        self.U[2:N+2, 0] = self.U[2:N+2, 1]
        self.V[2:N+2, 0] = self.V[2:N+2, 1]

        # Top
        self.U[1:N+2, M + 1] = 2. - self.U[1:N+2, M]
        self.V[1:N+2, M] = 0.
        self.U[1:N+2, M + 2] = self.U[1:N+2, M + 1]
        self.V[1:N+2, M + 1] = self.V[1:N+2, M]

    def Load_Pressure_Boundary(self, N, M):
        self.P[1, 1:M+2] = self.P[2, 1:M+2]
        self.P[N+1, 1:M+2] = self.P[N, 1:M+2]
        self.P[2:N+2, 1] = self.P[2:N+2, 2]
        self.P[2:N+2, M+1] = self.P[2:N+2, M]

    def QUICK(self, N, M):
        FUU = np.zeros((N + 3, M + 3))
        FVV = np.zeros((N + 3, M + 3))

        for i in range(2, N):
            for j in range(2, M + 1):
                X1 = self.DX[i] / 2.
                X2 = self.DX[i + 1] / 2.
                X3 = self.DX[i + 1] + self.DX[i + 2] / 2.
                X4 = self.DX[i - 1] + self.DX[i] / 2.
                X5 = self.DX[i] / 2.
                X6 = self.DX[i - 1] / 2.
                X7 = self.DX[i - 1] + self.DX[i - 2] / 2.
                X8 = self.DX[i] + self.DX[i + 1] / 2.

                UE = 0.5 * (self.DX[i] * self.U[i + 1][j] / self.DX1[i] + self.DX[i + 1] * self.U[i][j] / self.DX1[i])
                if UE > 0.:
                    A1 = X2 * X4 / (X4 - X1) / (X1 + X2)
                    A2 = X1 * X4 / (X1 + X2) / (X2 + X4)
                    A3 = -X1 * X2 / (X4 - X1) / (X2 + X4)
                    UE1 = A1 * self.U[i][j] + A2 * self.U[i + 1][j] + A3 * self.U[i - 1][j]
                else:
                    A1 = -X1 * X3 / (X1 + X2) / (X2 - X3)
                    A2 = X2 * X3 / (X1 + X3) / (X1 + X2)
                    A3 = X1 * X2 / (X1 + X3) / (X2 - X3)
                    UE1 = A1 * self.U[i + 1][j] + A2 * self.U[i][j] + A3 * self.U[i + 2][j]

                UW = 0.5 * (self.DX[i] * self.U[i - 1][j] / self.DX1[i - 1] + self.DX[i - 1] * self.U[i][j] / self.DX1[i - 1])
                if UW > 0.:
                    A1 = X5 * X7 / (X7 - X6) / (X5 + X6)
                    A2 = X6 * X7 / (X5 + X7) / (X5 + X6)
                    A3 = -X5 * X6 / (X7 - X6) / (X5 + X7)
                    UW1 = A1 * self.U[i - 1][j] + A2 * self.U[i][j] + A3 * self.U[i - 2][j]
                else:
                    A1 = X6 * X8 / (X5 + X6) / (X8 - X5)
                    A2 = X5 * X8 / (X5 + X6) / (X6 + X8)
                    A3 = -X5 * X6 / (X6 + X8) / (X8 - X5)
                    UW1 = A1 * self.U[i][j] + A2 * self.U[i - 1][j] + A3 * self.U[i + 1][j]

                #print(UE, UE1, UW, UW1)
                FUX = (UE * UE1 - UW * UW1) / self.DX[i]

                Y1 = self.DY[j] / 2.
                Y2 = self.DY[j] / 2.
                Y3 = self.DY[j + 1] + self.DY[j] / 2.
                Y4 = self.DY[j - 1] + self.DY[j] / 2.
                Y5 = self.DY[j - 1] / 2.
                Y6 = self.DY[j - 1] / 2.
                Y7 = self.DY[j - 2] + self.DY[j - 1] / 2.
                Y8 = self.DY[j] + self.DY[j - 1] / 2.

                VN = 0.5 * (self.V[i][j] + self.V[i + 1][j])
                if VN > 0.:
                    A1 = Y2 * Y4 / (Y4 - Y1) / (Y1 + Y2)
                    A2 = Y1 * Y4 / (Y1 + Y2) / (Y2 + Y4)
                    A3 = -Y1 * Y2 / (Y4 - Y1) / (Y2 + Y4)
                    UNN = A1 * self.U[i][j] + A2 * self.U[i][j + 1] + A3 * self.U[i][j - 1]
                else:
                    A1 = -Y1 * Y3 / (Y1 + Y2) / (Y2 - Y3)
                    A2 = Y2 * Y3 / (Y1 + Y3) / (Y1 + Y2)
                    A3 = Y1 * Y2 / (Y1 + Y3) / (Y2 - Y3)
                    UNN = A1 * self.U[i][j + 1] + A2 * self.U[i][j] + A3 * self.U[i][j + 2]

                VS = 0.5 * (self.V[i][j - 1] + self.V[i + 1][j - 1])
                if VS > 0.:
                    A1 = Y5 * Y7 / (Y7 - Y6) / (Y5 + Y6)
                    A2 = Y6 * Y7 / (Y5 + Y7) / (Y5 + Y6)
                    A3 = -Y5 * Y6 / (Y7 - Y6) / (Y5 + Y7)
                    US1 = A1 * self.U[i][j - 1] + A2 * self.U[i][j] + A3 * self.U[i][j - 2]
                else:
                    A1 = Y6 * Y8 / (Y5 + Y6) / (Y8 - Y5)
                    A2 = Y5 * Y8 / (Y5 + Y6) / (Y6 + Y8)
                    A3 = -Y5 * Y6 / (Y6 + Y8) / (Y8 - Y5)
                    US1 = A1 * self.U[i][j] + A2 * self.U[i][j - 1] + A3 * self.U[i][j + 1]

                FUY = (VN * UNN - VS * US1) / self.DY1[j - 1]

                FUC = 0.
                VDX = (self.U[i + 1][j] - self.U[i][j]) / self.DX[i] - (self.U[i][j] - self.U[i - 1][j]) / self.DX[i - 1]
                VDX = VDX / self.DX1[i - 1]
                VDY = (self.U[i][j + 1] - self.U[i][j]) / self.DY1[j] - (self.U[i][j] - self.U[i][j - 1]) / self.DY1[j - 1]
                VDY = VDY / self.DY[j]
                VISX = (VDX + VDY) / self.RE
                FUU[i][j] = self.DT *((self.P[i][j] - self.P[i + 1][j])/ self.DX[i] - FUX - FUY - FUC + VISX)

        for i in range(2, N + 1):
            for j in range(2, M):
                X1 = self.DX[i] / 2.
                X2 = self.DX[i] / 2.
                X3 = self.DX[i + 1] + self.DX[i] / 2.
                X4 = self.DX[i - 1] + self.DX[i] / 2.
                X5 = self.DX[i - 1] / 2.
                X6 = self.DX[i - 1] / 2.
                X7 = self.DX[i - 2] + self.DX[i - 1] / 2.
                X8 = self.DX[i] + self.DX[i - 1] / 2.

                UE = 0.5 * (self.U[i][j] + self.U[i][j + 1])
                if UE > 0.:
                    A1 = X2 * X4 / (X4 - X1) / (X1 + X2)
                    A2 = X1 * X4 / (X1 + X2) / (X2 + X4)
                    A3 = -X1 * X2 / (X4 - X1) / (X2 + X4)
                    VE1 = A1 * self.V[i][j] + A2 * self.V[i + 1][j] + A3 * self.V[i - 1][j]
                else:
                    A1 = -X1 * X3 / (X1 + X2) / (X2 - X3)
                    A2 = X2 * X3 / (X1 + X3) / (X1 + X2)
                    A3 = X1 * X2 / (X1 + X3) / (X2 - X3)
                    VE1 = A1 * self.V[i + 1][j] + A2 * self.V[i][j] + A3 * self.V[i + 2][j]

                UW = 0.5 * (self.U[i - 1][j] + self.U[i - 1][j + 1])
                if UW > 0.:
                    A1 = X5 * X7 / (X7 - X6) / (X5 + X6)
                    A2 = X6 * X7 / (X5 + X7) / (X5 + X6)
                    A3 = -X5 * X6 / (X7 - X6) / (X5 + X7)
                    VW1 = A1 * self.V[i - 1][j] + A2 * self.V[i][j] + A3 * self.V[i - 2][j]
                else:
                    A1 = X6 * X8 / (X5 + X6) / (X8 - X5)
                    A2 = X5 * X8 / (X5 + X6) / (X6 + X8)
                    A3 = -X5 * X6 / (X6 + X8) / (X8 - X5)
                    VW1 = A1 * self.V[i][j] + A2 * self.V[i - 1][j] + A3 * self.V[i + 1][j]

                FVX = (UE * VE1 - UW * VW1) / self.DX1[i - 1]
                Y1 = self.DY[j] / 2.
                Y2 = self.DY[j + 1] / 2.
                Y3 = self.DY[j + 1] + self.DY[j + 2] / 2.
                Y4 = self.DY[j - 1] + self.DY[j] / 2.
                Y5 = self.DY[j] / 2.
                Y6 = self.DY[j - 1] / 2.
                Y7 = self.DY[j - 1] + self.DY[j - 2] / 2.
                Y8 = self.DY[j] + self.DY[j + 1] / 2.

                VN = 0.5 * (self.DY[j + 1] * self.V[i][j] / self.DY1[j] + self.DY[j] * self.V[i][j + 1] / self.DY1[j])
                if VN > 0.:
                    A1 = Y2 * Y4 / (Y4 - Y1) / (Y1 + Y2)
                    A2 = Y1 * Y4 / (Y1 + Y2) / (Y2 + Y4)
                    A3 = -Y1 * Y2 / (Y4 - Y1) / (Y2 + Y4)
                    VNN = A1 * self.V[i][j] + A2 * self.V[i][j + 1] + A3 * self.V[i][j - 1]
                else:
                    A1 = -Y1 * Y3 / (Y1 + Y2) / (Y2 - Y3)
                    A2 = Y2 * Y3 / (Y1 + Y3) / (Y1 + Y2)
                    A3 = Y1 * Y2 / (Y1 + Y3) / (Y2 - Y3)
                    VNN = A1 * self.V[i][j + 1] + A2 * self.V[i][j] + A3 * self.V[i][j + 2]

                VS = 0.5 * (self.DY[j] * self.V[i][j - 1] / self.DY1[j - 1] + self.DY[j - 1] * self.V[i][j] / self.DY1[j - 1])
                if VS > 0.:
                    A1 = Y5 * Y7 / (Y7 - Y6) / (Y5 + Y6)
                    A2 = Y6 * Y7 / (Y5 + Y7) / (Y5 + Y6)
                    A3 = -Y5 * Y6 / (Y7 - Y6) / (Y5 + Y7)
                    VS1 = A1 * self.V[i][j - 1] + A2 * self.V[i][j] + A3 * self.V[i][j - 2]
                else:
                    A1 = Y6 * Y8 / (Y5 + Y6) / (Y8 - Y5)
                    A2 = Y5 * Y8 / (Y5 + Y6) / (Y6 + Y8)
                    A3 = -Y5 * Y6 / (Y6 + Y8) / (Y8 - Y5)
                    VS1 = A1 * self.V[i][j] + A2 * self.V[i][j - 1] + A3 * self.V[i][j + 1]

                FVY = (VN * VNN - VS * VS1) / self.DY[j]
                FVC = 0.
                VDX = (self.V[i + 1][j] - self.V[i][j]) / self.DX[i] - (self.V[i][j] - self.V[i - 1][j]) / self.DX[i - 1]
                VDX = VDX / self.DX1[i - 1]
                VDY = (self.V[i][j + 1] - self.V[i][j]) / self.DY1[j] - (self.V[i][j] - self.V[i][j - 1]) / self.DY1[j - 1]
                VDY = VDY / self.DY[j]
                VISY = (VDX + VDY) / self.RE
                FVV[i][j] = self.DT *((self.P[i][j] - self.P[i][j+1])/self.DY[j] - FVX - FVY - FVC + VISY)

        return FUU, FVV

    def Solver(self, N, M):
        Main_Name = 'Cavity_Flow'
        TIME = 0.
        REIT = 1
        TOL = 1e-4

        FUUN1 = np.zeros((N + 3, M + 3))
        FUUN2 = np.zeros((N + 3, M + 3))
        FVVN1 = np.zeros((N + 3, M + 3))
        FVVN2 = np.zeros((N + 3, M + 3))
        #Fx = np.zeros((M + 3, N + 3))
        #Fy = np.zeros((M + 3, N + 3))
        #DP = np.zeros((M + 3, N + 3))

        if self.RERUN == 1:
            File_Holder = open('LAST.DAT', "r")
            Temp_All_Lines = File_Holder.readlines()
            TIME = float(Temp_All_Lines[0])
            REIT = float(Temp_All_Lines[1])

            i = 1
            j = 1
            Total = (N + 1) * ( M + 1)
            for k in range(Total):
                Splitted_Lines = Temp_All_Lines[2+k].split()
                self.U[i][j] = float(Splitted_Lines[0])
                self.V[i][j] = float(Splitted_Lines[1])
                self.P[i][j] = float(Splitted_Lines[2])

                Splitted_Lines = Temp_All_Lines[2+k+Total].split()
                FUUN1[i][j] = float(Splitted_Lines[0])
                FUUN2[i][j] = float(Splitted_Lines[1])
                FVVN1[i][j] = float(Splitted_Lines[2])
                FVVN2[i][j] = float(Splitted_Lines[3])

                j += 1
                if j == M+2:
                    j = 1
                    i += 1

            REIT += 1
            File_Holder.close()

        for TITE in range(REIT, self.TTT + 1):
            TIME += self.DT
            print("Time = " + str(TIME))
            # self.VOS(N, M, self.Cylinder_Center, self.Cylinder_Radius)

            FUU, FVV = self.QUICK(N, M)

            # 1 = First order (Euler), 2 = second order (Adam Bashforth), 3 = third order (Adam Bashforth)
            if REIT == 1:
                self.U[2: N, 2: M + 1] += FUU[2: N, 2: M + 1]
                self.V[2: N + 1, 2: M] += FVV[2: N + 1, 2: M]
            elif REIT == 2:
                self.U[2: N, 2: M + 1] += 1.5 * FUU[2: N, 2: M + 1] - 0.5 * FUUN1[2: N, 2: M + 1]
                self.V[2: N + 1, 2: M] += 1.5 * FVV[2: N + 1, 2: M]- 0.5 * FVVN1[2: N + 1, 2: M]
            else:
                self.U[2: N, 2: M + 1] += (23. * FUU[2: N, 2: M + 1] - 16. * FUUN1[2: N, 2: M + 1] + 5. * FUUN2[2: N, 2: M + 1]) / 12.
                self.V[2: N + 1, 2: M] += (23. * FVV[2: N + 1, 2: M] - 16. * FVVN1[2: N + 1, 2: M] + 5. * FVVN2[2: N + 1, 2: M]) / 12.
            FUUN2[2: N, 2: M + 1] = np.copy(FUUN1[2: N, 2: M + 1])
            FUUN1[2: N, 2: M + 1] = np.copy(FUU[2: N, 2: M + 1])
            FVVN2[2: N + 1, 2: M] = np.copy(FVVN1[2: N + 1, 2: M])
            FVVN1[2: N + 1, 2: M] = np.copy(FVV[2: N + 1, 2: M])

            self.Load_Vel_Boundary(N, M)

            for PPPP in range(1, self.ITER + 1):
                DMAX = 0.
                for i in range(2, N + 1):
                    for j in range(2, M + 1):
                        DD = (self.U[i, j] - self.U[i - 1, j]) / self.DX[i] \
                             + (self.V[i, j] - self.V[i, j - 1]) /self.DY[j]
                        DP = -DD/self.DT/\
                             ((1./self.DX1[i] + 1./self.DX1[i-1])/self.DX[i] +
                              (1./self.DY1[j] + 1./self.DY1[j-1])/self.DY[j])
                        self.P[i, j] += self.OMEGA * DP
                        self.U[i, j] += self.DT * DP /self.DX1[i]
                        self.U[i-1, j] += - self.DT * DP/self.DX1[i-1]
                        self.V[i, j] += self.DT * DP /self.DY1[j]
                        self.V[i, j-1] += - self.DT * DP/self.DY1[j-1]

                        if abs(DD) > DMAX:
                            DMAX = abs(DD)

                #self.Load_Pressure_Boundary(N, M)
                self.Load_Vel_Boundary(N, M)
                if (DMAX < TOL):
                    break
            print('DMAX= ' + str(DMAX))

            for i in range(2, N):
                for j in range(2, M + 1):
                    self.Fx[i, j] = self.Eta[i, j] * (self.USolid - self.U[i, j])/ self.DT
                    self.U[i, j] += self.Eta[i, j] * (self.USolid - self.U[i, j])

            for i in range(2, N + 1):
                for j in range(2, M):
                    self.Fy[i, j] = self.Eta[i, j] * (self.VSolid - self.V[i, j])/ self.DT
                    self.V[i, j] += self.Eta[i, j] * (self.VSolid - self.V[i, j])

            self.Load_Vel_Boundary(N, M)

            #UN1[1:N+2, 1:M+2] = np.copy(self.U[1:N+2, 1:M+2])
            #VN1[1:N+2, 1:M+2] = np.copy(self.V[1:N+2, 1:M+2])
            #TS = np.sqrt(TS / 2. / (N + 1) / (M + 1))   #total data is the same

            print('UV-P ITERATION NUMBER =' + str(PPPP))
            #print('DMAX= ' + str(DMAX) + ' ,I= '+ str(II) + ' ,J= '+ str(JJ))
            #print('THE MAX. VARIATION=' + str(TEMP) + ' ,SQRT OF VARIATION=' + str(TS))

            if TITE % self.FT == 0:
                File_Holder = open('flow.dat', "a")
                File_Holder.writelines(str(TITE)+" "+str(TIME)+" "+str(PPPP)+" "+str(DMAX) +"\n")
                File_Holder.close()

            Name_Digits = 5
            if TITE % self.OT == 0:
                File_Name = Main_Name + "_TS-"
                Log10_Value = int(np.floor(np.log10(TITE)))
                for i in range(Name_Digits-Log10_Value-1):
                    File_Name = File_Name + "0"
                File_Name = File_Name + str(TITE)

                #ST = np.zeros((M + 3, N + 3))
                #VORT = np.zeros((M + 3, N + 3))

                #for i in range(2, N + 1):
                #    for j in range(2, M + 1):
                #        ST[i][j] = ST[i][j-1] + (self.Y[j+1]-self.Y[j]) * self.U[i][j]
                #        VORT[i][j] = -(self.U[i-1][j] - self.U[i-1][j-1])/(0.5 * (self.Y[j+1] - self.Y[j-1])) \
                #                     +(self.V[i][j-1] - self.V[i-1][j-1])/(0.5 * (self.X[i+1] - self.X[i-1]))

                File_Holder = open(File_Name, "w+")
                #File_Holder.writelines('TITLE="CAVITY FLOW"\n')
                #File_Holder.writelines('VARIABLES=X,Y,U,V,P,STREAM,VORTICITY\n')
                #File_Holder.writelines('ZONE T="CAVITY FLOW", I='+ str(N) + ' , J='+ str(M) + ' F=POINT\n')
                for j in range(1, M+2):
                    for i in range(1, N+2):
                        x_Middle = (self.X[i] + self.X[i+1])/2.
                        y_Middle = (self.Y[j] + self.Y[j+1])/2.
                        File_Holder.writelines(str(x_Middle)+ " "+str(y_Middle)+" " \
                                               +str(self.U[i,j])+" "+str(self.V[i,j])+" "+str(self.Eta[i,j])+" "+str(self.P[i,j])+" " \
                                               +str(self.Fx[i,j])+" "+str(self.Fy[i,j])+"\n")
                File_Holder.close()

            REFILE = 'RERUN.DAT'
            File_Holder = open(REFILE, "w+")
            File_Holder.writelines(str(TIME) + "\n")
            File_Holder.writelines(str(TITE) + "\n")
            for i in range(1, N + 2):
                for j in range(1, M+2):
                    File_Holder.writelines(str(self.U[i][j])+" "+str(self.V[i][j])+" "+str(self.P[i][j])+"\n")
            for i in range(1, N + 2):
                for j in range(1, M+2):
                    File_Holder.writelines(str(FUUN1[i][j])+" "+str(FUUN2[i][j])+" "+str(FVVN1[i][j])+" "+str(FVVN2[i][j])+"\n")
            File_Holder.close()


In [4]:
# Main Function
MC = Main_Class(N, M, Cylinder_Center, Cylinder_Radius, uSolid)
t1 = time()
MC.Load_Input(N, M)
#MC.Load_Vel_Boundary(N, M)

MC.Solver(N, M)
t2 = time()
print("Time taken by code was {:.10f}" .format(t2 - t1) + " seconds")
File = open('CLOCK.dat', "w+")
File.writelines('Time taken by code was {:.10f}' .format(t2 - t1) + ' seconds\n')
File.close()
print("The program (Solver) has run sucessfully (Pass)")

Time = 0.001
DMAX= 0.0
UV-P ITERATION NUMBER =1
Time = 0.002
DMAX= 0.1481360841983117
UV-P ITERATION NUMBER =100
Time = 0.003
DMAX= 0.06172474065865936
UV-P ITERATION NUMBER =100
Time = 0.004
DMAX= 0.03617618043146434
UV-P ITERATION NUMBER =100
Time = 0.005
DMAX= 0.038616328704629055
UV-P ITERATION NUMBER =100
Time = 0.006
DMAX= 0.03726449156652062
UV-P ITERATION NUMBER =100
Time = 0.007
DMAX= 0.03072886242207218
UV-P ITERATION NUMBER =100
Time = 0.008
DMAX= 0.022882446145100807
UV-P ITERATION NUMBER =100
Time = 0.009000000000000001
DMAX= 0.024538539305670966
UV-P ITERATION NUMBER =100
Time = 0.010000000000000002
DMAX= 0.02348287533833159
UV-P ITERATION NUMBER =100
Time = 0.011000000000000003
DMAX= 0.016402446249520725
UV-P ITERATION NUMBER =100
Time = 0.012000000000000004
DMAX= 0.009088925042739674
UV-P ITERATION NUMBER =100
Time = 0.013000000000000005
DMAX= 0.009320982913665411
UV-P ITERATION NUMBER =100
Time = 0.014000000000000005
DMAX= 0.011935155556819
UV-P ITERATION NUMBER =100
T

KeyboardInterrupt: 